In [6]:
!nvidia-smi

Tue Jan 14 01:49:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   60C    P8             19W /  133W |    1708MiB /   6144MiB |     18%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
import torch
from torchvision import models
from torchvision import transforms

In [10]:
from timeit import default_timer as timer
from typing import Tuple, Dict
import random
from pathlib import Path
import matplotlib.pyplot as plt

In [11]:
# Import/install Gradio
try:
    import gradio as gr
except:
    !pip -q install gradio
    import gradio as gr

print(f"Gradio version: {gr.__version__}")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires keras>=3.2.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.17.0 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.10.0 which is incompatible.


Gradio version: 5.12.0


In [12]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [13]:
# model = models.efficientnet_b0(pretrained=True)

In [14]:
# # Move the model to GPU if available
# model = model.to(device)
# # model.device

In [15]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

# Define data transformations for EfficientNet-B0
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet-B0 default input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# Load datasets
train_dataset = ImageFolder('output_dataset/train', transform=train_transforms)
val_dataset = ImageFolder('output_dataset/train', transform=val_transforms)

# train_dataset.to(device)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [22]:
# Classes in the dataset
print(f"Classes: {train_dataset.classes}")

class_names=train_dataset.classes
class_names

num_classes=len(class_names)
num_classes

Classes: ['Battery', 'Keyboard', 'Microwave', 'Mobile', 'Mouse', 'PCB', 'Player', 'Printer', 'Television', 'Washing Machine']


10

In [2]:
# Display some sample images
def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    plt.imshow(img.permute(1, 2, 0))
    plt.show()

In [9]:
# data_iter = iter(train_loader)
# images, labels = next(data_iter)

# # Show the first image in the batch
# imshow(images[7])
# print(f"Label: {train_dataset.classes[labels[7]]}")

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.558952..1.5934205].


: 

In [18]:
# model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [20]:
# model = torch.load("model_weights/efficientnet_b0_e_waste_ep_15_10_classes.pth", map_location=torch.device('cpu'))
# model.load_state_dict(state_dict)

# state_dict = torch.load("model_weights/efficientnet_b0_e_waste_ep_15_10_classes.pth", map_location=torch.device('cpu'))
# model.load_state_dict(state_dict)

In [21]:
# model.load_state_dict(torch.load('model_weights/efficientnet_b0_e_waste_ep_15_10_classes.pth'))

In [4]:
import torch
from torchvision.models import efficientnet_b0

# Step 1: Define the model architecture
num_classes = 10  # Replace with the number of classes in your dataset
model = efficientnet_b0(pretrained=False)  # Same architecture as used during training
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)

# Step 2: Load the state dictionary
state_dict = torch.load("model_weights/efficientnet_b0_e_waste_ep_15_10_classes.pth", map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# Step 3: Set the model to evaluation mode (for inference)
model.eval()

print("Model loaded successfully!")


Model loaded successfully!


C:\Users\alok4\AppData\Local\Temp\ipykernel_47732\3523833913.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("model_weights/efficientnet_b0_e_wa

In [5]:
model.state_dict()

OrderedDict([('features.0.0.weight',
              tensor([[[[ 1.2260e-01,  6.5678e-01,  4.5696e-01],
                        [-1.1013e-01, -6.0977e-01, -3.3325e-01],
                        [ 2.8789e-02, -1.0273e-01, -1.0287e-01]],
              
                       [[ 6.3884e-02,  1.6548e+00,  1.7429e+00],
                        [-1.3629e-01, -1.5371e+00, -1.5943e+00],
                        [ 5.0483e-02, -1.1375e-01, -1.2580e-01]],
              
                       [[ 8.7595e-02,  3.6098e-01,  2.6921e-01],
                        [-1.1960e-01, -2.8167e-01, -2.1917e-01],
                        [ 3.6776e-02, -7.0962e-02, -8.1952e-02]]],
              
              
                      [[[ 1.6431e-01, -2.0058e-01,  8.3010e-02],
                        [ 8.8901e-01, -1.2121e+00,  2.7468e-01],
                        [ 1.0728e+00, -1.2616e+00,  2.0467e-01]],
              
                       [[ 3.2816e-01, -4.3447e-01,  1.8781e-01],
                        [ 1.6204e+00, 

In [6]:
# Put EffNetB2 on CPU
model.to("cpu")

# Check the device
next(iter(model.parameters())).device

device(type='cpu')

In [18]:
# Get a list of all test image filepaths
test_data_paths = list(Path("").glob("*/*.jpeg"))
test_data_paths

[WindowsPath('examples/1images.jpeg'), WindowsPath('examples/2images.jpeg')]

In [19]:
# Create a list of example inputs to our Gradio demo
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=0)]
example_list

[]

In [23]:
def predict(img) -> Tuple[Dict, float]:
    """Transforms and performs a prediction on img and returns prediction and time taken.
    """
    # Start the timer
    start_time = timer()

    # Transform the target image and add a batch dimension
    img = val_transforms(img).unsqueeze(0)
    # img.to(device)
    # Put model into evaluation mode and turn on inference mode
    model.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_probs = torch.softmax(model(img), dim=1)

    # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

    # Calculate the prediction time
    pred_time = round(timer() - start_time, 5)

    # Return the prediction dictionary and prediction time
    return pred_labels_and_probs, pred_time

In [24]:
import gradio as gr

# Create title, description and article strings
title = "E-Waste 🍕🥩🍣"
description = "An EfficientNetB2 feature extractor computer vision model to classify images of food as pizza, steak or sushi."
article = "Created at [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/)."

# Create the Gradio demo
demo = gr.Interface(fn=predict, # mapping function from input to output
                    inputs=gr.Image(type="pil"), # what are the inputs?
                    outputs=[gr.Label(num_top_classes=10, label="Predictions"), # what are the outputs?
                             gr.Number(label="Prediction time (s)")], # our fn has two outputs, therefore we have two outputs
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)

# Launch the demo!
demo.launch(debug=True, # print errors locally?
            share=True) # generate a publically shareable URL?

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Keyboard interruption in main thread... closing server.
